In [1]:
import time
import praw
from datetime import datetime
import threading
import pandas as pd

In [2]:
reddit = praw.Reddit(client_id='***', client_secret="***",
                     password='password', user_agent='***',
                     username='username')

In [3]:
class Post:
    commentforrest = reddit.submission('3hahrw').comments
    comment = []
    upvote_count = 0
    def __init__(self, subreddit, title, created_time,Pid):
        self.subreddit = subreddit
        self.title = title
        self.created_time = created_time
        self.Pid = Pid

In [4]:
class Comment:
    def __init__(self, body,Cid):
        self.Cid = Cid
        self.body = body

In [5]:
submissions = []
hots = []

In [6]:
subreddits = [reddit.subreddit('CoronavirusUS'),reddit.subreddit('CoronavirusUK'),reddit.subreddit('CoronavirusNewYork'),reddit.subreddit('CoronavirusCA'),reddit.subreddit('CoronavirusMichigan'),reddit.subreddit('CoronavirusPA'),reddit.subreddit('CoronavirusIllinois')]

In [7]:
def submissionstream(subreddits,submissions):#stream#
    string = str("")
    for i in range(0,len(subreddits)):
        if (i != 0):
            string += "+"+str(subreddits[i].title)
        else:
            string += str(subreddits[i].title)
    for submission in reddit.subreddit(string).stream.submissions():
        print(str(submission.subreddit.title) + "; " + submission.title+"; "+str(datetime.utcfromtimestamp(submission.created_utc))+";"+str(submission.score))
        dt = datetime.utcfromtimestamp(submission.created_utc)
        post = Post(submission.subreddit.title, submission.title, dt, submission.id)
        submissions.append(post)
    return submissions

In [18]:
def submissionsupdate(subreddits,submissions):#called at the end of the day#
    for submission in submissions:
        post = reddit.submission(id=submission.Pid)
        submission.upvote_count = post.score
        submission.commentforrest = post.comments
        for comment in submission.commentforrest:
            if isinstance(comment, praw.models.MoreComments):
                continue
            submission.comment.append(Comment(comment.body, comment.id))
    print("update_done!")
    return submissions

In [9]:
def hot(subreddits,hots):
    for sub in subreddits:
        hotposts = sub.hot(limit = 50)
        for submission in hotposts:
            dt = datetime.utcfromtimestamp(submission.created_utc)
            post = Post(submission.subreddit.title, submission.title, dt, submission.id)
            post.upvote_count = submission.score
            hots.append(post)
        print(sub.title+" done!")
    return hots, datetime.utcnow()

In [10]:
def csvwriter(submissions,hots):
    stream_dic = {"subreddit":[],
                  "Pid":[],
                  "title":[],
                  "upvote":[],
                  "date":[]}
    comment_dic = {"Cid":[],
                   "Pid":[],
                   "body":[]}
    hot_dic = {"subreddit":[],
               "Pid":[],
               "title":[],
               "upvote":[],
               "date":[]}
    for post in submissions:
        stream_dic["subreddit"].append(post.subreddit)
        stream_dic["Pid"].append(post.Pid)
        stream_dic["title"].append(post.title)
        stream_dic["upvote"].append(post.upvote_count)
        stream_dic["date"].append(post.created_time)
        for comment in post.comment:
            comment_dic["Cid"].append(comment.Cid)
            comment_dic["Pid"].append(post.Pid)
            comment_dic["body"].append(comment.body)
    for post in hots:
        hot_dic["subreddit"].append(post.subreddit)
        hot_dic["Pid"].append(post.Pid)
        hot_dic["title"].append(post.title)
        hot_dic["upvote"].append(post.upvote_count)
        hot_dic["date"].append(post.created_time)
    dt = str(datetime.utcnow()).split(" ")[0]
    df_stream = pd.DataFrame(stream_dic)
    df_comment = pd.DataFrame(comment_dic)
    df_hot = pd.DataFrame(hot_dic)
    df_stream.to_csv(str('stream'+dt+'.csv'))
    df_comment.to_csv(str('comment'+dt+'.csv'))
    df_hot.to_csv(str('hot'+dt+'.csv'))

In [16]:
submissionstream(subreddits,submissions)

CoronavirusNewYork; Very important coronavirus last updates NewYork 12-04; 2020-04-13 18:11:13;0
CoronavirusUS; As of 4/13 only 2.5% of small business applicants have received loans from the paycheck protection program; 2020-04-13 18:11:53;6
CoronavirusCA; Police shut down SF nightclub operating during coronavirus crisis; 2020-04-13 18:15:58;59
CoronavirusUS; CDC director says COVID-19 cases have 'stabilized,' will soon start to decline; 2020-04-13 18:16:29;10
CoronavirusUK; Can I go for another job whilst being on Furlough pay from my part time?!; 2020-04-13 18:21:12;2
CoronavirusUS; Florida jurisdictions have stopped reporting C19 deaths.; 2020-04-13 18:21:42;10
CoronavirusCA; Report: Stockpile of 39 million masks exposed as fake | KRON4; 2020-04-12 23:32:31;6
CoronavirusCA; How the coronavirus crisis gave Gov. Gavin Newsom his leadership moment; 2020-04-12 22:14:41;12
CoronavirusCA; Hospitalization rates in SD County are far below capacity for now; 2020-04-12 21:37:14;3
CoronavirusC

KeyboardInterrupt: 

In [ ]:
submissionsupdate(subreddits,submissions)

In [24]:
hot(subreddits,hots)

CoronavirusUS done!
CoronavirusUK done!
CoronavirusNewYork done!
CoronavirusCA done!
Talk about Coronavirus in Michigan  done!
CoronaVirusPA done!
CoronavirusIllinois done!
CoronaVirusFlorida done!
CoronavirusIndiana done!


([<__main__.Post at 0x127b75ef208>,
 datetime.datetime(2020, 4, 13, 6, 50, 12, 469058))

In [25]:
csvwriter(submissions,hots)

In [14]:
for i in submissions:
    print(i)